In [ ]:
import pandas as pd
import requests
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

class AddressGeocodingTool:
    def __init__(self):
        self.api_key = None
        self.headers = None
        self.address_cache = {}

    def set_api_key(self, key):
        self.api_key = key
        self.headers = {"Authorization": f"KakaoAK {key}"}

    def geocoding(self, address):
        if not isinstance(address, str) or not address.strip():
            return {"lat": None, "lng": None}
        
        if address in self.address_cache:
            return self.address_cache[address]
        
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        params = {"query": address}
        
        try:
            response = requests.get(url, headers=self.headers, params=params, timeout=5)
            response.raise_for_status()
            data = response.json()
            
            if data['documents']:
                first_result = data['documents'][0]
                crd = {"lat": str(first_result['y']), "lng": str(first_result['x'])}
                self.address_cache[address] = crd
                return crd
            else:
                self.address_cache[address] = {"lat": None, "lng": None}
                return {"lat": None, "lng": None}
        except Exception as e:
            self.address_cache[address] = {"lat": None, "lng": None}
            return {"lat": None, "lng": None}

    def run(self):
        api_input = input("API KEY 입력: ").strip()
        self.set_api_key(api_input)

        file_path = input("해당 코드와 파일을 같은 디렉토리에 넣은 뒤 확장자 포함 파일명 입력: ").strip()
        if not os.path.exists(file_path):
            print(f"'{file_path}' 파일을 찾을 수 없습니다.")
            return

        ext = os.path.splitext(file_path)[1].lower()
        
        try:
            if ext in ['.xlsx', '.xls']:
                print("모든 시트(월별 데이터)를 읽어와 하나로 합치는 중입니다...")
                sheets_dict = pd.read_excel(file_path, sheet_name=None)
                all_dfs = list(sheets_dict.values())
                df = pd.concat(all_dfs, ignore_index=True)
                
            elif ext == '.json':
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                if isinstance(data, dict):
                    data = data.get('DATA', data)
                df = pd.DataFrame(data)
            else:
                print("지원하지 않는 파일 형식.")
                return
                
            print(f"\n파일을 정상적으로 불러왔습니다. (총 {len(df)}행)")
            print("--- 컬럼 목록 ---")
            for idx, col in enumerate(df.columns):
                print(f"{idx}: {col}")
            print("----------------")
        except Exception as e:
            print(f"파일 로드 중 오류 발생: {e}")
            return

        print("\n 주소 데이터로 사용할 컬럼의 번호를 입력해 주십시오.")
        print("(예: '3, 1' 처럼 시군구, 동/번지 컬럼 번호를 순서대로 입력)")
        col_input = input("   입력: ").strip()

        try:
            target_indices = [int(x.strip()) for x in col_input.split(',') if x.strip().isdigit()]
            
            if not target_indices:
                print("오류: 유효한 숫자가 입력되지 않았습니다.")
                return

            selected_cols = [df.columns[i] for i in target_indices]
            print(f"선택된 컬럼 데이터를 병합: {selected_cols}")

            df['__merged_address__'] = df[selected_cols].apply(
                lambda row: ' '.join(row.values.astype(str)), axis=1
            )
            # nan 문자열 제거 및 공백 정리
            df['__merged_address__'] = df['__merged_address__'].str.replace('nan', '', regex=False).str.strip()
            print(f"(미리보기) '{df['__merged_address__'].iloc[0]}'")

        except (ValueError, IndexError) as e:
            print(f"컬럼 선택 오류: ({e})")
            return

        unique_addresses = df['__merged_address__'].dropna().unique()
        unique_addresses = [addr for addr in unique_addresses if addr.strip() != ""]
        
        print(f"\n좌표 변환을 시작합니다. (고유 주소 {len(unique_addresses)}건)")
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_address = {executor.submit(self.geocoding, addr): addr for addr in unique_addresses}
            for _ in tqdm(as_completed(future_to_address), total=len(unique_addresses), desc="진행률"):
                pass

        print("\n데이터 병합 및 정규화 중...")
        # 원본 데이터프레임에 좌표 매핑
        df['lat'] = df['__merged_address__'].map(lambda addr: self.address_cache.get(addr, {}).get('lat'))
        df['lng'] = df['__merged_address__'].map(lambda addr: self.address_cache.get(addr, {}).get('lng'))
        
        # 임시 주소 컬럼 삭제
        df.drop(columns=['__merged_address__'], inplace=True)

        # 유효한 좌표가 있는 행만 정규화 진행
        valid_mask = df['lat'].notnull() & df['lng'].notnull()
        if valid_mask.sum() > 0:
            df.loc[valid_mask, 'lat'] = pd.to_numeric(df.loc[valid_mask, 'lat'])
            df.loc[valid_mask, 'lng'] = pd.to_numeric(df.loc[valid_mask, 'lng'])
            
            scaler = MinMaxScaler()
            scaled = scaler.fit_transform(df.loc[valid_mask, ['lat', 'lng']])
            
            df.loc[valid_mask, 'lat_scaled'] = scaled[:, 0]
            df.loc[valid_mask, 'lng_scaled'] = scaled[:, 1]

        output_filename = os.path.splitext(file_path)[0] + "_processed" + ext
        if ext == '.json':
            df.to_json(output_filename, orient='records', force_ascii=False, indent=4)
        else:
            df.to_excel(output_filename, index=False)
            
        print(f"\n모든 작업이 완료되었습니다. 결과 파일: '{output_filename}'")

if __name__ == "__main__":
    tool = AddressGeocodingTool()
    tool.run()
    input("\n종료하려면 아무 키나 누르세요.")

모든 시트(월별 데이터)를 읽어와 하나로 합치는 중입니다...

파일을 정상적으로 불러왔습니다. (총 31695행)
--- 컬럼 목록 ---
0: Unnamed: 0
1: Unnamed: 1
2: Unnamed: 2
3: Unnamed: 3
4: Unnamed: 4
5: Unnamed: 5
----------------

 주소 데이터로 사용할 컬럼의 번호를 입력해 주십시오.
(예: '3, 1' 처럼 시군구, 동/번지 컬럼 번호를 순서대로 입력)
선택된 컬럼 데이터를 병합: ['Unnamed: 3']
(미리보기) '주소'

좌표 변환을 시작합니다. (고유 주소 11247건)


진행률: 100%|██████████| 11247/11247 [12:11<00:00, 15.38it/s]



데이터 병합 및 정규화 중...

모든 작업이 완료되었습니다. 결과 파일: '2025_1_7_processed.xlsx'
